In [1]:
%load_ext autoreload
%autoreload 2

# `AnalyticalSolver`

In [ ]:
from plugins.boyan_chain.analytical_solver import AnalyticalSolverBoyanChain

/Users/richardweiss/Documents/.venvs/dice_rl_TU_Vienna/lib/python3.11/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <
/Users/richardweiss/Documents/.venvs/dice_rl_TU_Vienna/lib/python3.11/site-packages/tf_agents/__init__.py:58: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if distutils.version.LooseVersion(
/Users/richardweiss/Documents/.venvs/dice_rl_TU_Vienna/lib/python3.11/site-packages/tf_agents/utils/common.py:91: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  distutils.version.LooseVersion(tf.__version__)


In [ ]:
def test(N, p, gamma):
    kind = "episodic" if gamma < 1 else "continuing"

    analytical_solver = AnalyticalSolverBoyanChain(N=N, p=p, kind=kind)

    pv, sdc, info = analytical_solver.solve(gamma, primal_dual="dual") # type: ignore

    sd_d = analytical_solver.dD
    sd_p = sd_d * sdc

    analytical_solver.test(gamma=gamma, sd=sd_p)

In [4]:
for gamma in [0.1, 0.5, 0.9]:
    test(N=12, p=0.1, gamma=gamma)

Checking backwards Bellman equations (gamma=0.1):
-------------------------------------------------
d^pi = (1 - gamma) * d_0^pi + gamma * P^pi_* d^pi
-> MSE = 2.2424815307994407e-35
d^pi >= 0
-> True
sum(d^pi) = 1
-> sum = 1.0

Checking backwards Bellman equations (gamma=0.5):
-------------------------------------------------
d^pi = (1 - gamma) * d_0^pi + gamma * P^pi_* d^pi
-> MSE = 5.792836160852233e-35
d^pi >= 0
-> True
sum(d^pi) = 1
-> sum = 0.9999999999999999

Checking backwards Bellman equations (gamma=0.9):
-------------------------------------------------
d^pi = (1 - gamma) * d_0^pi + gamma * P^pi_* d^pi
-> MSE = 1.0886886141461801e-36
d^pi >= 0
-> True
sum(d^pi) = 1
-> sum = 0.9999999999999992



In [5]:
test(N=12, p=0.1, gamma=1.0)

Checking backwards Bellman equations (gamma=1.0):
-------------------------------------------------
d^pi = P^pi_* d^pi
-> MSE = 1.3020064782163243e-33
d^pi >= 0
-> True
sum(d^pi) = 1
-> sum = 0.9999999999999999



# OHE

In [ ]:
import os

from dice_rl_TU_Vienna.dataset import load_or_create_dataset_Experience, one_hot_encode_observation, print_env_step
from plugins.boyan_chain.environment import get_env

In [ ]:
data_dir = os.path.join("data", "boyan_chain")

datasets_dir = os.path.join(data_dir, "datasets")
outputs_dir  = os.path.join(data_dir, "outputs")

seed = 0
num_experience = 100_000
p = 0.1
N = 12

k = "c"
kind = "continuing"

hparam_str_dataset = f"{seed=}_{num_experience=}_{N=}_kind={kind}"
dataset_dir = os.path.join(datasets_dir, hparam_str_dataset)

In [ ]:
env = get_env(seed=seed, N=N, kind=kind)

dataset = load_or_create_dataset_Experience(
    dataset_dir=dataset_dir,
    env=env, num_experience=num_experience, seed=seed,
)

data/boyan_chain/datasets/seed=0_num_experience=100000_N=12_kind=continuing/dataset-ctr.pkl


In [ ]:
env_step, _ = dataset.get_episode()

In [ ]:
env_step_OHC_obs = one_hot_encode_observation(env_step, dataset, dataset)

In [ ]:
print_env_step(env_step)
print_env_step(env_step_OHC_obs)

,step_type,step_num,observation,action,reward,discount
0,0,0,2,0,-3.0,1.0
1,1,1,6,1,-3.0,1.0
2,1,2,4,0,-3.0,1.0


,step_type,step_num,observation,action,reward,discount
0,0,0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,-3.0,1.0
1,1,1,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]",1,-3.0,1.0
2,1,2,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",0,-3.0,1.0


# `aux_recorder`

In [ ]:
from dice_rl_TU_Vienna.runners.aux_recorders import aux_recorder_cos_angle
from dice_rl_TU_Vienna.runners.neural_dual_dice_runner import NeuralDualDiceRunner
from dice_rl_TU_Vienna.dataset import one_hot_encode_observation

from plugins.boyan_chain.continuous.config import *
from plugins.boyan_chain.continuous.load import *

getting experience


100%|██████████| 100000/100000 [00:13<00:00, 7224.56it/s]


adding experience to dataset


100%|██████████| 100000/100000 [06:09<00:00, 270.71it/s]


getting experience


100%|██████████| 100000/100000 [00:13<00:00, 7151.08it/s]


adding experience to dataset


100%|██████████| 100000/100000 [06:00<00:00, 277.74it/s]


data/boyan_chain/datasets/seed=1_num_experience=100000_N=12_kind=episodic/dataset-ctr.pkl
data/boyan_chain/datasets/seed=1_num_experience=100000_N=12_kind=continuing/dataset-ctr.pkl
data/boyan_chain/datasets/seed=2_num_experience=100000_N=12_kind=episodic/dataset-ctr.pkl
data/boyan_chain/datasets/seed=2_num_experience=100000_N=12_kind=continuing/dataset-ctr.pkl
data/boyan_chain/datasets/seed=3_num_experience=100000_N=12_kind=episodic/dataset-ctr.pkl
data/boyan_chain/datasets/seed=3_num_experience=100000_N=12_kind=continuing/dataset-ctr.pkl


In [48]:
neural_dual_dice_runner = NeuralDualDiceRunner(
    gamma=0.9,
    num_steps=100_000,
    batch_size=64,
    seed=0,
    primal_hidden_dims=(32,),
    dual_hidden_dims=(32,),
    primal_learning_rate=0.001,
    dual_learning_rate=0.001,
    regularizer_primal=0.0,
    regularizer_dual=0.0,
    f_exponent=1.5,
    dataset=dataset[0]["e"],
    dataset_spec=dataset_spec,
    target_policy=target_policy,
    save_dir=None,
    by="experience",
    analytical_solver=analytical_solver["e"],
    env_step_preprocessing=one_hot_encode_observation,
    aux_recorder=aux_recorder_cos_angle,
    aux_recorder_pbar=["cos_angle"],
)

NeuralDualDice
gam0.9_batchs64_seed0_hdp(32,)_hdd(32,)_lrp0.001_lrd0.001_regp0.0_regd0.0_fexp1.5


100%|██████████| 100000/100000 [44:10<00:00, 37.73it/s, loss=69.3, pv_s=-1.14, pv_w=-1.02, cos_angle=0.902, pv_error_s=0.325, pv_error_w=0.201, sdc_L2_error=0.0433]    
